In [10]:
import numpy as np
import pandas as pd

In [51]:
users = pd.read_csv('../../data/Users.csv', delimiter=';')
books = pd.read_csv('../../data/Books.csv', delimiter=';', dtype={'ISBN': str, 'Title': str, 'Author': str, 'Year': np.int16, 'Publisher': str})
ratings = pd.read_csv('../../data/Ratings.csv', delimiter=';', dtype={'User-ID': np.int32, 'ISBN': str, 'Rating': np.int8})

/var/folders/yn/16crnc792zg2yr78zf0qg0j40000gq/T/ipykernel_37212/2231409931.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('../../data/Users.csv', delimiter=';')


In [12]:
import sys
sys.path.append('../..')

from src.evaluation import average_precision
from sklearn.model_selection import train_test_split

In [13]:
train_ratings, test_ratings = train_test_split(ratings, test_size=0.3, random_state=42)

In [52]:
from src.recommenders.base import BaseRecommender
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

n_recomm = 5

books = books.reset_index() # add index as a column
isbn_mapping = {category: idx for idx, category in enumerate(books['ISBN'])}

ratings = ratings.copy()
ratings['ISBN_i'] = ratings['ISBN'].map(isbn_mapping) # map ISBN to index
ratings.dropna(subset=['ISBN_i'], inplace=True) # drop rows with NaN ISBN_i
ratings['ISBN_i'] = ratings['ISBN_i'].astype(np.int32)

# Create a sparse user-item matrix
user_item_matrix = csr_matrix((ratings['Rating'], (ratings['User-ID'], ratings['ISBN_i'])), dtype=np.float64)

# Normalize the user-item matrix
normalized_matrix = user_item_matrix.copy()
means = np.array([normalized_matrix[i].data.mean() for i in range(normalized_matrix.shape[0])])
normalized_matrix.data -= np.repeat(means, np.diff(normalized_matrix.indptr))

item_user_matrix = csr_matrix((ratings['Rating'], (ratings['ISBN_i'], ratings['User-ID'])), dtype=np.float64)
similarity_matrix = cosine_similarity(normalized_matrix.T, dense_output=False)

print('Similarity matrix shape:', similarity_matrix.shape)
print('Item-user matrix shape:', item_user_matrix.shape)
print('User-to-item matrix shape:', user_item_matrix.shape)



/var/folders/yn/16crnc792zg2yr78zf0qg0j40000gq/T/ipykernel_37212/1133943471.py:22: RuntimeWarning: Mean of empty slice.
  means = np.array([normalized_matrix[i].data.mean() for i in range(normalized_matrix.shape[0])])
/Users/oleh.sannikov/miniconda3/envs/recsys/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Similarity matrix shape: (271379, 271379)
Item-user matrix shape: (271379, 278855)
User-to-item matrix shape: (278855, 271379)


In [53]:
def list_minus(a, b):
    return [x for x in a if x not in b]

def predict(users, items):
    user_predictions = {}
    for user_id in users:
        # Get the row corresponding to the user
        user_row = normalized_matrix.getrow(user_id)
        top_rated = np.argsort(user_row.data)[::-1][:n_recomm]

        recommended_books = []
        for book_i in top_rated:
            book_row = similarity_matrix[book_i]
            most_similar_books = np.argsort(book_row.data)[::-1][:n_recomm]
            recommended_books.extend(most_similar_books)

        recommended_books = list_minus(set(recommended_books), user_row.indices)
        print(recommended_books)
        average_ratings = {}
        for book in recommended_books:
            # Get the average rating for the book
            average_rating = 0 #normalized_matrix.getcol(book).data.mean()
            average_ratings[book] = average_rating

        sorted_recommended_books = sorted(recommended_books, key=lambda book: average_ratings[book], reverse=True)[:n_recomm]

        user_predictions[user_id] = books[books['ISBN'].isin(sorted_recommended_books)]
    
    return user_predictions

In [56]:
test_users = test_ratings['User-ID'].unique()[:100]
len(test_users)

100

In [25]:
normalized_matrix.shape

(278855, 271379)

In [47]:
similarity_matrix.shape

(278855, 278855)

In [26]:
books.shape

(271379, 6)

In [57]:
best_items = predict(test_users, items=books)

[16264, 37515, 5644, 8858, 4517, 1063, 1448, 16170, 16431, 4798, 16323, 11204, 969, 1353, 129611, 1101, 973, 12896, 7654, 5610, 618, 4590, 7667, 5619, 5624]
[0, 1, 2, 387, 16516, 4, 3, 23437, 3863, 794, 29728, 6314, 6319, 6328, 322, 327, 739, 740, 741, 7406, 1648, 369, 47097]
[0, 1, 2, 3, 4, 1671, 1677, 23, 5798, 5804, 5810, 26163, 3252, 3255, 3256, 3257, 3385, 16056, 12611, 5337, 16605, 15229]
[0, 1, 2, 4, 5, 1924, 6, 8, 1289, 9, 10, 13, 11150, 15, 1294, 1317, 2342, 10549, 54, 12099, 73, 79, 87, 13025, 1402]
[13025, 73, 79, 54, 87]
[14, 50090, 5292, 5302, 41152, 30658, 968, 20424, 21067, 972, 974, 14671, 976, 23383, 42972, 30307, 34406, 1516, 46706, 14969]
[1, 1218, 34, 13025, 1959, 73, 1260, 1230, 1454, 79, 51, 54, 87, 25, 27]
[1, 23074, 3, 4122, 2, 11526, 4, 35304, 12649, 24904, 8, 12, 11, 5, 18130, 4085, 6680, 6522]
[]
[0, 1, 9, 16140, 20, 22, 23, 14492, 11175, 7084, 7090, 13878, 8887, 8383, 22114, 1514, 1519, 2550, 7035, 7038, 1535]
[23808, 21626, 21650, 27157, 23715, 49445, 18855

In [ ]:
len(best_items)
best_items[38781]['ISBN']

73871    0671000403
57042    0373078358
31483    0425176940
2243     0345348109
50150    0441644511
Name: ISBN, dtype: object

In [55]:
from tqdm import tqdm

rating_mask = test_ratings['Rating'] >= 5

mean_ap = 0
for i in tqdm(range(len(test_users))):
    user_mask = test_ratings['User-ID'] == test_users[i]
    relevant_items = test_ratings.loc[(user_mask & rating_mask)].sort_values(by='Rating', ascending=False)
    if len(relevant_items) == 0:
        continue
    user_id = test_users[i]
    ap = average_precision(best_items[user_id]['ISBN'].to_numpy(), relevant_items['ISBN'].to_numpy())
    mean_ap += ap

mean_ap /= len(test_users)
mean_ap

100%|██████████| 1/1 [00:00<00:00, 20.43it/s]


0.0